In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:/Users/HP/OneDrive/Chatbot/project.json').read()
intents = json.loads(data_file)



for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('C:/Users/HP/OneDrive/Chatbot/words.pkl','wb'))
pickle.dump(classes,open('C:/Users/HP/OneDrive/Chatbot/classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

77 documents
18 classes ['Ranking', 'about', 'club', 'courses', 'cse', 'exam', 'fee', 'goodbye', 'greeting', 'griet', 'location', 'moodle', 'options', 'phno', 'placements', 'results', 'scholarships', 'thanks']
91 unique lemmatized words ["'s", ',', 'about', 'activity', 'and', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'club', 'college', 'company', 'computer', 'contact', 'could', 'course', 'cse', 'curricular', 'day', 'detail', 'do', 'engineering', 'exam', 'fee', 'for', 'good', 'goodbye', 'griet', 'group', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'is', 'know', 'later', 'located', 'location', 'many', 'me', 'moodle', 'name', 'next', 'nice', 'non', 'notif', 'number', 'of', 'offered', 'phno', 'phone', 'placement', 'provided', 'rank', 'ranking', 'reimbursement', 'result', 'scholarship', 'science', 'see', 'show', 'statistic', 'structure', 'subject', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'up', 'visited', 'w

In [4]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('C:/Users/HP/OneDrive/Chatbot/chatbot_model.h5', hist)

print("model created")

Epoch 1/200
77/77 [==============================] - 2s 25ms/step - loss: 2.9414 - accuracy: 0.0390
Epoch 2/200
77/77 [==============================] - 0s 727us/step - loss: 2.8470 - accuracy: 0.1039
Epoch 3/200
77/77 [==============================] - 0s 688us/step - loss: 2.7710 - accuracy: 0.1039
Epoch 4/200
77/77 [==============================] - 0s 727us/step - loss: 2.6856 - accuracy: 0.1688
Epoch 5/200
77/77 [==============================] - 0s 623us/step - loss: 2.5882 - accuracy: 0.1948
Epoch 6/200
77/77 [==============================] - 0s 623us/step - loss: 2.4633 - accuracy: 0.2597
Epoch 7/200
77/77 [==============================] - 0s 623us/step - loss: 2.4116 - accuracy: 0.3117
Epoch 8/200
77/77 [==============================] - 0s 623us/step - loss: 2.3055 - accuracy: 0.3117
Epoch 9/200
77/77 [==============================] - 0s 831us/step - loss: 2.1051 - accuracy: 0.3896
Epoch 10/200
77/77 [==============================] - 0s 1ms/step - loss: 2.0067 - accuracy:

77/77 [==============================] - 0s 519us/step - loss: 0.0860 - accuracy: 1.0000
Epoch 82/200
77/77 [==============================] - 0s 519us/step - loss: 0.1981 - accuracy: 0.9481
Epoch 83/200
77/77 [==============================] - 0s 519us/step - loss: 0.1408 - accuracy: 0.9610
Epoch 84/200
77/77 [==============================] - 0s 519us/step - loss: 0.1582 - accuracy: 0.9481
Epoch 85/200
77/77 [==============================] - 0s 519us/step - loss: 0.1289 - accuracy: 0.9870
Epoch 86/200
77/77 [==============================] - 0s 416us/step - loss: 0.1053 - accuracy: 1.0000
Epoch 87/200
77/77 [==============================] - 0s 416us/step - loss: 0.0853 - accuracy: 1.0000
Epoch 88/200
77/77 [==============================] - 0s 416us/step - loss: 0.1166 - accuracy: 0.9740
Epoch 89/200
77/77 [==============================] - 0s 519us/step - loss: 0.0885 - accuracy: 1.0000
Epoch 90/200
77/77 [==============================] - 0s 519us/step - loss: 0.1057 - accuracy: 

77/77 [==============================] - 0s 519us/step - loss: 0.0566 - accuracy: 1.0000
Epoch 161/200
77/77 [==============================] - 0s 416us/step - loss: 0.0848 - accuracy: 0.9870
Epoch 162/200
77/77 [==============================] - 0s 416us/step - loss: 0.0728 - accuracy: 0.9870
Epoch 163/200
77/77 [==============================] - 0s 519us/step - loss: 0.0810 - accuracy: 0.9740
Epoch 164/200
77/77 [==============================] - 0s 416us/step - loss: 0.1016 - accuracy: 0.9610
Epoch 165/200
77/77 [==============================] - 0s 519us/step - loss: 0.0604 - accuracy: 0.9740
Epoch 166/200
77/77 [==============================] - 0s 519us/step - loss: 0.1089 - accuracy: 0.9610
Epoch 167/200
77/77 [==============================] - 0s 416us/step - loss: 0.0206 - accuracy: 1.0000
Epoch 168/200
77/77 [==============================] - 0s 519us/step - loss: 0.1184 - accuracy: 0.9610
Epoch 169/200
77/77 [==============================] - 0s 519us/step - loss: 0.0448 - a